In [18]:
# ===============================
# Jupyter Notebook: EMS Metaheuristics (Mashhad EMS)
# ===============================

# ======= Section 0: Imports =======
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import random
import time

# ======= Section 1: Load & Preprocess Mashhad EMS Dataset =======
import pandas as pd

# Remplacer par ton chemin complet
admissions = pd.read_csv(r"C:\Users\amelb\OneDrive\Bureau\Mashhad_dataset\ED_admission.csv")
triage = pd.read_csv(r"C:\Users\amelb\OneDrive\Bureau\Mashhad_dataset\ED_triage.csv")


print("Colonnes ED_admission.csv :", admissions.columns.tolist())
print("Colonnes ED_triage.csv :", triage.columns.tolist())


Colonnes ED_admission.csv : ['triage_code', 'PatientCode', 'DischargeFromED', 'Foreigners', 'gender', 'age', 'marital_Status', 'ResidentDate_year', 'ResidentDate_month', 'ResidentDate_day', 'ResidentDate_weekday', 'ResidentDate_hour', 'DischargeDate_year', 'DischargeDate_month', 'DischargeDate_day', 'DischargeDate_weekday', 'DischargeDate_hour', 'StatusOnDischarge', 'KindRef', 'IsGlobal', 'AccompainerRelation_id', 'ResidentDay', 'service_count_action', 'service_count_instrument', 'service_count_lab', 'service_count_graphy']
Colonnes ED_triage.csv : ['triage_code', 'gender', 'age', 'admission_year', 'admission_month', 'admission_day', 'admission_weekday', 'admission_hour', 'kindref', 'ChiefComplaint', 'explainer_id', 'NeedFastExecute', 'CriticalStatus', 'StuporStatus', 'PainGrade', 'MentalDistress', 'MaterialDistress', 'Source', 'BlooddpressurSystol', 'BlooddpressurDiastol', 'PulseRate', 'RespiratoryRate', 'Temperature', 'O2Saturation', 'AVPU', 'TriageGrade', 'operational_patient', 'ref

In [19]:
# Fusionner sur la colonne commune 'triage_code'
df = pd.merge(admissions, triage, on="triage_code")

# Vérifier le résultat
df.head()

,triage_code,PatientCode,DischargeFromED,Foreigners,gender_x,age_x,marital_Status,ResidentDate_year,ResidentDate_month,ResidentDate_day,...,BlooddpressurSystol,BlooddpressurDiastol,PulseRate,RespiratoryRate,Temperature,O2Saturation,AVPU,TriageGrade,operational_patient,ref_specialist
0,13960101008,9600008,0,No,Female,75.0,2.0,2017,3,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0,0
1,13960101009,9600009,0,No,Male,41.0,2.0,2017,3,21,...,NaN,NaN,86.0,18.0,NaN,96.0,A,3,0,0
2,13960101010,9600010,0,No,Female,71.0,2.0,2017,3,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0
3,13960101011,9600011,0,No,Male,77.0,2.0,2017,3,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0
4,13960101012,9600012,0,No,Male,38.0,2.0,2017,3,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1,0


In [20]:
print(admissions.columns)

Index(['triage_code', 'PatientCode', 'DischargeFromED', 'Foreigners', 'gender',
       'age', 'marital_Status', 'ResidentDate_year', 'ResidentDate_month',
       'ResidentDate_day', 'ResidentDate_weekday', 'ResidentDate_hour',
       'DischargeDate_year', 'DischargeDate_month', 'DischargeDate_day',
       'DischargeDate_weekday', 'DischargeDate_hour', 'StatusOnDischarge',
       'KindRef', 'IsGlobal', 'AccompainerRelation_id', 'ResidentDay',
       'service_count_action', 'service_count_instrument', 'service_count_lab',
       'service_count_graphy'],
      dtype='object')


In [2]:
!pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/22.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/22.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/22.9 MB ? eta -:--:--
    --------------------------------------- 0.5/22.9 MB 1.1 MB/s eta 0:00:21
   - -------------------------------------- 0.8/22.9 MB 1.1 MB/s eta 0:00:20
   - -------------------------------------- 1.0/22.9 MB 1.1 MB/s eta 0:00:21
   -- ------------------------------------- 1.3/22.9 MB 1.1 MB/s eta 0:00:20
   -- ------------------------------------- 1.3/22.9 MB 1.1 MB/s eta 0:00:20
   -- ------------------------------------- 1.6/22.9 MB 1.1 MB/s eta 0:00:20
   --- ------------------------------------ 1.8/22.9 MB 1.1 MB/s eta 0:00:20
   --- ------------------------------------ 2.1/22.9 MB 1.1 MB/s eta 0:00:20
   ---- ----------------------------------- 2.4/22.9 MB 1.1 MB/s eta 0:00:20
   ---- -----------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# =========================================================
# Mashhad EMS ARP Metaheuristics - Medium Dataset
# =========================================================

# ======= Section 0: Imports =======
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, LineString
from scipy.spatial.distance import cdist
import random
import copy
import matplotlib.pyplot as plt
import time 

# ======= Section 1: Chemins =======
base_path = r"C:\Users\amelb\OneDrive\Bureau\MashhadDataset\\"

# ======= Section 2: Charger les données =======
calls = pd.read_excel(base_path + "Data_file_1_Mashhad_EMS_Calls.xlsx")
census_tracts = gpd.read_file(base_path + "02 Mashhad_Census_Tracts\\Mashhad_Census_Tracts.shp")
ambulance_stations = gpd.read_file(base_path + "03 Mashhad_Ambulance_Stations\\Mashhad_Ambulance_Stations.shp")

# ======= Section 3: Prétraitement spatial =======
# Centroids des tracts (projeter en UTM pour éviter warnings)
census_tracts = census_tracts.to_crs(epsg=32640)
census_tracts["centroid"] = census_tracts.geometry.centroid

# Fusion avec les appels EMS
calls = calls.merge(
    census_tracts[['Tract_ID', 'centroid']],
    left_on='Census tract ID',
    right_on='Tract_ID',
    how='left'
)

# Créer GeoDataFrame
calls['geometry'] = calls['centroid']
calls_gdf = gpd.GeoDataFrame(calls, geometry="geometry", crs="EPSG:32640")

# Supprimer appels sans géométrie
calls_gdf = calls_gdf.dropna(subset=["geometry"]).reset_index(drop=True)

# Réduire dataset pour test
MAX_CALLS = 3000
if len(calls_gdf) > MAX_CALLS:
    calls_gdf = calls_gdf.sample(MAX_CALLS, random_state=42).reset_index(drop=True)

stations_gdf = ambulance_stations.to_crs(epsg=32640)
stations_coords = np.array([[pt.x, pt.y] for pt in stations_gdf.geometry])
calls_coords = np.array([[pt.x, pt.y] for pt in calls_gdf.geometry])

print("Nombre d'appels EMS utilisés :", len(calls_gdf))

# ======= Section 4: Matrices de distances =======
dist_station_call = cdist(stations_coords, calls_coords)
dist_call_call = cdist(calls_coords, calls_coords)

# Paramètres ARP
N_calls = len(calls_gdf)
N_ambulances = len(stations_gdf)
service_time = np.full(N_calls, 10.0)        # minutes
AMBULANCE_SPEED_KMH = 40
METERS_PER_MIN = (AMBULANCE_SPEED_KMH * 1000) / 60
# Critical time réaliste
critical_time = np.full(N_calls, 60.0)      # minutes

# ======= Section 5: Initialisation de solutions =======
def init_solution():
    calls_idx = list(range(N_calls))
    random.shuffle(calls_idx)
    sol = [[] for _ in range(N_ambulances)]
    for i, c in enumerate(calls_idx):
        sol[i % N_ambulances].append(c)
    return sol

# ======= Section 6: Evaluation / contraintes =======
def evaluate(solution, w1=1.0, w2=0.5, w3=5.0):
    total_distance = 0.0
    response_times = []
    coverage = 0

    for k, route in enumerate(solution):
        if not route:
            continue

        # station → premier appel
        dist = dist_station_call[k, route[0]]
        time = dist / METERS_PER_MIN

        total_distance += dist
        response_times.append(time)

        if time <= critical_time[route[0]]:
            coverage += 1

        # appels successifs
        for i in range(len(route) - 1):
            dist = dist_call_call[route[i], route[i+1]]
            travel_time = dist / METERS_PER_MIN

            time += travel_time + service_time[route[i]]
            total_distance += dist
            response_times.append(time)

            if time <= critical_time[route[i+1]]:
                coverage += 1

    if len(response_times) == 0:
        return np.inf

    avg_response = np.mean(response_times)
    return w1*(total_distance/1000) + w2*avg_response - w3*coverage

def compute_metrics(solution):
    total_distance = 0
    response_times = []
    coverage = 0
    for k, route in enumerate(solution):
        if not route:
            continue
        dist = dist_station_call[k, route[0]]
        time = dist / METERS_PER_MIN
        total_distance += dist
        response_times.append(time)
        if time <= critical_time[route[0]]:
            coverage += 1
        for i in range(len(route)-1):
            dist = dist_call_call[route[i], route[i+1]]
            travel_time = dist / METERS_PER_MIN
            time += travel_time + service_time[route[i]]
            total_distance += dist
            response_times.append(time)
            if time <= critical_time[route[i+1]]:
                coverage += 1
    avg_response = np.mean(response_times) if response_times else 0
    return total_distance, avg_response, coverage

# ======= Section 7: Genetic Algorithm (GA) =======
def GA(pop_size=20, generations=10):
    population = [init_solution() for _ in range(pop_size)]
    best_sol = population[0]
    best_eval = evaluate(best_sol)

    for g in range(generations):
        population.sort(key=evaluate)
        new_pop = population[:4]  # élitisme
        while len(new_pop) < pop_size:
            p1, p2 = random.sample(population[:10], 2)
            cut = random.randint(1, N_ambulances-1)
            child = p1[:cut] + p2[cut:]
            new_pop.append(copy.deepcopy(child))
        population = new_pop
        best_sol = population[0]
        best_eval = evaluate(best_sol)
        print(f"GA Gen {g+1} | Best Eval = {best_eval:.2f}")
    return best_sol

# ======= Section 8: Tabu Search (TS) =======
def TS(initial, iterations=50, tabu_size=20, w1=1.0, w2=0.5, w3=0.1):
    """
    Tabu Search pour Ambulance Routing Problem
    - initial : solution initiale (liste de routes)
    - iterations : nombre d'itérations
    - tabu_size : taille de la liste tabou
    - w1, w2, w3 : poids pour evaluate()
    """
    current = copy.deepcopy(initial)
    best = copy.deepcopy(current)
    tabu = []

    for it in range(iterations):
        neighbors = []

        for k in range(N_ambulances):
            # 1) Permuter deux appels dans la même ambulance
            if len(current[k]) > 1:
                nb = copy.deepcopy(current)
                i, j = random.sample(range(len(nb[k])), 2)
                nb[k][i], nb[k][j] = nb[k][j], nb[k][i]
                neighbors.append(nb)

            # 2) Déplacer un appel vers une autre ambulance
            if len(current[k]) > 0:
                from_k = k
                to_k = random.choice([x for x in range(N_ambulances) if x != from_k])
                nb = copy.deepcopy(current)
                idx = random.randint(0, len(nb[from_k]) - 1)
                call = nb[from_k].pop(idx)
                nb[to_k].append(call)
                neighbors.append(nb)

        # Filtrer les solutions taboues
        neighbors = [n for n in neighbors if n not in tabu]
        if not neighbors:
            break

        # Sélectionner le meilleur voisin
        current = min(neighbors, key=lambda sol: evaluate(sol, w1=w1, w2=w2, w3=w3))
        if evaluate(current, w1=w1, w2=w2, w3=w3) < evaluate(best, w1=w1, w2=w2, w3=w3):
            best = copy.deepcopy(current)

        # Mise à jour de la liste tabou
        tabu.append(current)
        if len(tabu) > tabu_size:
            tabu.pop(0)

        print(f"TS Iter {it+1} | Best Eval = {evaluate(best, w1=w1, w2=w2, w3=w3):.2f}")

    return best


# ======= Section 9: Simulated Annealing (SA) =======
def SA(initial, T0=500, alpha=0.95, iterations=50, w1=1.0, w2=0.5, w3=0.1):
    """
    Simulated Annealing pour Ambulance Routing Problem
    - initial : solution initiale
    - T0 : température initiale
    - alpha : facteur de décroissance
    - iterations : nombre d'itérations
    - w1, w2, w3 : poids pour evaluate()
    """
    current = copy.deepcopy(initial)
    best = copy.deepcopy(current)
    T = T0

    for it in range(iterations):
        k = random.randint(0, N_ambulances-1)
        new = copy.deepcopy(current)

        if len(new[k]) > 1 and random.random() < 0.5:
            # Permutation dans la même ambulance
            i, j = random.sample(range(len(new[k])), 2)
            new[k][i], new[k][j] = new[k][j], new[k][i]
        elif len(new[k]) > 0:
            # Déplacement vers une autre ambulance
            to_k = random.choice([x for x in range(N_ambulances) if x != k])
            idx = random.randint(0, len(new[k]) - 1)
            call = new[k].pop(idx)
            new[to_k].append(call)
        else:
            continue  # rien à faire

        delta = evaluate(new, w1=w1, w2=w2, w3=w3) - evaluate(current, w1=w1, w2=w2, w3=w3)

        if delta < 0 or random.random() < np.exp(-delta / T):
            current = new

        if evaluate(current, w1=w1, w2=w2, w3=w3) < evaluate(best, w1=w1, w2=w2, w3=w3):
            best = copy.deepcopy(current)

        T *= alpha
        print(f"SA Iter {it+1} | Best Eval = {evaluate(best, w1=w1, w2=w2, w3=w3):.2f}")

    return best


# ======= Section 10: Ant Colony Optimization (ACO) =======
def ACO(ants=10, iterations=10, rho=0.1):
    pheromone = np.ones((N_calls, N_calls))
    best_sol = init_solution()
    best_eval = evaluate(best_sol)

    for it in range(iterations):
        solutions = []
        for _ in range(ants):
            sol = [[] for _ in range(N_ambulances)]
            calls_idx = list(range(N_calls))
            random.shuffle(calls_idx)
            for c in calls_idx:
                k = random.randint(0, N_ambulances-1)
                sol[k].append(c)
            solutions.append(sol)
        for sol in solutions:
            cost = evaluate(sol)
            if cost < best_eval:
                best_eval = cost
                best_sol = copy.deepcopy(sol)
            for r in sol:
                for i in range(len(r)-1):
                    pheromone[r[i], r[i+1]] = (1-rho)*pheromone[r[i], r[i+1]] + rho/cost
        print(f"ACO Iter {it+1} | Best Eval = {best_eval:.2f}")
    return best_sol

# ======= Section 11: Exécution avec temps =======
solutions = {}
metrics = {}
comp_time = {}
# Initial solution
init = init_solution()

# Genetic Algorithm
start = time.perf_counter()
best_ga = GA()
end = time.perf_counter()
solutions["GA"] = best_ga
comp_time["GA"] = end - start

# Tabu Search
start = time.perf_counter()
best_ts = TS(init)
end = time.perf_counter()
solutions["TS"] = best_ts
comp_time["TS"] = end - start

# Simulated Annealing
start = time.perf_counter()
best_sa = SA(init)
end = time.perf_counter()
solutions["SA"] = best_sa
comp_time["SA"] = end - start

# Ant Colony Optimization
start = time.perf_counter()
best_aco = ACO()
end = time.perf_counter()
solutions["ACO"] = best_aco
comp_time["ACO"] = end - start

# ======= Section 12: Calcul métriques avec temps =======
metrics_df = pd.DataFrame(columns=["Total Distance (km)", "Avg Response (min)", "Critical Coverage (calls)", "Computational Time (s)"])
for name, sol in solutions.items():
    total_dist, avg_resp, critical_cov = compute_metrics(sol)
    metrics_df.loc[name] = [total_dist/1000, avg_resp, critical_cov, comp_time[name]]

print("\n=== Métriques finales avec temps d'exécution ===")
print(metrics_df)

Nombre d'appels EMS utilisés : 3000
GA Gen 1 | Best Eval = 24549.48
GA Gen 2 | Best Eval = 24341.39
GA Gen 3 | Best Eval = 24341.39
GA Gen 4 | Best Eval = 24268.55
GA Gen 5 | Best Eval = 24157.55
GA Gen 6 | Best Eval = 24157.55
GA Gen 7 | Best Eval = 23981.47
GA Gen 8 | Best Eval = 23981.47
GA Gen 9 | Best Eval = 23981.47
GA Gen 10 | Best Eval = 23928.02
TS Iter 1 | Best Eval = 25795.03
TS Iter 2 | Best Eval = 25769.07
TS Iter 3 | Best Eval = 25747.26
TS Iter 4 | Best Eval = 25721.31
TS Iter 5 | Best Eval = 25686.89
TS Iter 6 | Best Eval = 25659.45
TS Iter 7 | Best Eval = 25637.47
TS Iter 8 | Best Eval = 25598.98
TS Iter 9 | Best Eval = 25572.27
TS Iter 10 | Best Eval = 25526.96
TS Iter 11 | Best Eval = 25504.13
TS Iter 12 | Best Eval = 25472.21
TS Iter 13 | Best Eval = 25426.47
TS Iter 14 | Best Eval = 25389.69
TS Iter 15 | Best Eval = 25366.62
TS Iter 16 | Best Eval = 25333.94
TS Iter 17 | Best Eval = 25292.44
TS Iter 18 | Best Eval = 25266.63
TS Iter 19 | Best Eval = 25233.85
TS Ite

In [13]:
# =========================================================
# Mashhad EMS ARP Metaheuristics - Small Dataset
# =========================================================

# ======= Section 0: Imports =======
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from scipy.spatial.distance import cdist
import random
import copy
import time

# ======= Section 1: Chemins =======
base_path = r"C:\Users\amelb\OneDrive\Bureau\MashhadDataset\\"

# ======= Section 2: Charger les données =======
calls = pd.read_excel(base_path + "Data_file_1_Mashhad_EMS_Calls.xlsx")
census_tracts = gpd.read_file(base_path + "02 Mashhad_Census_Tracts\\Mashhad_Census_Tracts.shp")
ambulance_stations = gpd.read_file(base_path + "03 Mashhad_Ambulance_Stations\\Mashhad_Ambulance_Stations.shp")

# ======= Section 3: Prétraitement spatial =======
census_tracts = census_tracts.to_crs(epsg=32640)
census_tracts["centroid"] = census_tracts.geometry.centroid

calls = calls.merge(
    census_tracts[['Tract_ID', 'centroid']],
    left_on='Census tract ID',
    right_on='Tract_ID',
    how='left'
)

calls['geometry'] = calls['centroid']
calls_gdf = gpd.GeoDataFrame(calls, geometry="geometry", crs="EPSG:32640")
calls_gdf = calls_gdf.dropna(subset=["geometry"]).reset_index(drop=True)

# Réduire dataset pour test
MAX_CALLS = 10
if len(calls_gdf) > MAX_CALLS:
    calls_gdf = calls_gdf.sample(MAX_CALLS, random_state=42).reset_index(drop=True)

stations_gdf = ambulance_stations.to_crs(epsg=32640)
stations_coords = np.array([[pt.x, pt.y] for pt in stations_gdf.geometry])
calls_coords = np.array([[pt.x, pt.y] for pt in calls_gdf.geometry])

print("Nombre d'appels EMS utilisés :", len(calls_gdf))

# ======= Section 4: Matrices de distances =======
dist_station_call = cdist(stations_coords, calls_coords)
dist_call_call = cdist(calls_coords, calls_coords)

# Paramètres ARP
N_calls = len(calls_gdf)
N_ambulances = 6   # ajustable
service_time = np.full(N_calls, 10.0)
AMBULANCE_SPEED_KMH = 40
METERS_PER_MIN = (AMBULANCE_SPEED_KMH * 1000) / 60
critical_time = np.full(N_calls, 60.0)

# ======= Section 5: Solution initiale =======
def fixed_initial_solution():
    sol = [[] for _ in range(N_ambulances)]
    for i, c in enumerate(range(N_calls)):
        sol[i % N_ambulances].append(c)
    return sol

init_solution_fixed = fixed_initial_solution()

# ======= Section 6: Evaluation et métriques =======
def evaluate(solution, w1=1.0, w2=0.5, w3=0.1):
    total_distance = 0
    response_times = []
    coverage = 0
    for k, route in enumerate(solution):
        if not route:
            continue
        dist = dist_station_call[k, route[0]]
        time_acc = dist / METERS_PER_MIN
        total_distance += dist
        response_times.append(time_acc)
        if time_acc <= critical_time[route[0]]:
            coverage += 1
        for i in range(len(route)-1):
            dist = dist_call_call[route[i], route[i+1]]
            travel_time = dist / METERS_PER_MIN
            time_acc += travel_time + service_time[route[i]]
            total_distance += dist
            response_times.append(time_acc)
            if time_acc <= critical_time[route[i+1]]:
                coverage += 1
    avg_response = np.mean(response_times) if response_times else 0
    # Fonction objectif pondérée
    return w1*(total_distance/1000) + w2*avg_response - w3*coverage

def compute_metrics(solution):
    total_distance = 0
    response_times = []
    coverage = 0
    for k, route in enumerate(solution):
        if not route:
            continue
        dist = dist_station_call[k, route[0]]
        time_acc = dist / METERS_PER_MIN
        total_distance += dist
        response_times.append(time_acc)
        if time_acc <= critical_time[route[0]]:
            coverage += 1
        for i in range(len(route)-1):
            dist = dist_call_call[route[i], route[i+1]]
            travel_time = dist / METERS_PER_MIN
            time_acc += travel_time + service_time[route[i]]
            total_distance += dist
            response_times.append(time_acc)
            if time_acc <= critical_time[route[i+1]]:
                coverage += 1
    avg_response = np.mean(response_times) if response_times else 0
    return total_distance, avg_response, coverage

# ======= Section 7: Métaheuristiques avec permutations et déplacements =======
def GA(solution, generations=10):
    pop = [copy.deepcopy(solution) for _ in range(5)]
    best = copy.deepcopy(solution)
    for g in range(generations):
        for i in range(len(pop)):
            # permutation aléatoire dans chaque ambulance
            for route in pop[i]:
                if len(route) > 1:
                    a,b = random.sample(range(len(route)),2)
                    route[a], route[b] = route[b], route[a]
            # déplacement aléatoire entre ambulances
            from_k = random.randint(0,N_ambulances-1)
            if len(pop[i][from_k])>0:
                to_k = random.choice([x for x in range(N_ambulances) if x!=from_k])
                idx = random.randint(0,len(pop[i][from_k])-1)
                call = pop[i][from_k].pop(idx)
                pop[i][to_k].append(call)
        # choisir le meilleur
        best = min(pop, key=evaluate)
    return best

def TS(solution, iterations=50, tabu_size=20):
    current = copy.deepcopy(solution)
    best = copy.deepcopy(solution)
    tabu = []
    for it in range(iterations):
        neighbors = []
        for k in range(N_ambulances):
            if len(current[k])>1:
                nb = copy.deepcopy(current)
                i,j = random.sample(range(len(nb[k])),2)
                nb[k][i], nb[k][j] = nb[k][j], nb[k][i]
                neighbors.append(nb)
            if len(current[k])>0:
                to_k = random.choice([x for x in range(N_ambulances) if x!=k])
                nb = copy.deepcopy(current)
                idx = random.randint(0,len(nb[k])-1)
                call = nb[k].pop(idx)
                nb[to_k].append(call)
                neighbors.append(nb)
        neighbors = [n for n in neighbors if n not in tabu]
        if neighbors:
            current = min(neighbors,key=evaluate)
            if evaluate(current)<evaluate(best):
                best = copy.deepcopy(current)
            tabu.append(current)
            if len(tabu)>tabu_size:
                tabu.pop(0)
    return best

def SA(solution, T0=500, alpha=0.95, iterations=50):
    current = copy.deepcopy(solution)
    best = copy.deepcopy(solution)
    T = T0
    for it in range(iterations):
        k = random.randint(0,N_ambulances-1)
        new = copy.deepcopy(current)
        if len(new[k])>1 and random.random()<0.5:
            i,j = random.sample(range(len(new[k])),2)
            new[k][i], new[k][j] = new[k][j], new[k][i]
        elif len(new[k])>0:
            to_k = random.choice([x for x in range(N_ambulances) if x!=k])
            idx = random.randint(0,len(new[k])-1)
            call = new[k].pop(idx)
            new[to_k].append(call)
        delta = evaluate(new) - evaluate(current)
        if delta<0 or random.random()<np.exp(-delta/T):
            current = new
        if evaluate(current)<evaluate(best):
            best = copy.deepcopy(current)
        T *= alpha
    return best

def ACO(solution, ants=10, iterations=10, rho=0.1):
    best = copy.deepcopy(solution)
    for it in range(iterations):
        for _ in range(ants):
            new_sol = copy.deepcopy(solution)
            for route in new_sol:
                if len(route)>1:
                    a,b = random.sample(range(len(route)),2)
                    route[a], route[b] = route[b], route[a]
            # déplacement aléatoire entre ambulances
            from_k = random.randint(0,N_ambulances-1)
            if len(new_sol[from_k])>0:
                to_k = random.choice([x for x in range(N_ambulances) if x!=from_k])
                idx = random.randint(0,len(new_sol[from_k])-1)
                call = new_sol[from_k].pop(idx)
                new_sol[to_k].append(call)
            if evaluate(new_sol)<evaluate(best):
                best = copy.deepcopy(new_sol)
    return best

# ======= Section 8: Exécution et mesure temps =======
solutions = {}
comp_time = {}

for name, algo in [("GA",GA),("TS",TS),("SA",SA),("ACO",ACO)]:
    start = time.perf_counter()
    best_sol = algo(init_solution_fixed)
    end = time.perf_counter()
    solutions[name] = best_sol
    comp_time[name] = end-start

# ======= Section 9: Calcul métriques =======
metrics_df = pd.DataFrame(columns=["Total Distance (km)","Avg Response (min)","Critical Coverage (calls)","Computational Time (s)"])
for name, sol in solutions.items():
    total_dist, avg_resp, critical_cov = compute_metrics(sol)
    metrics_df.loc[name] = [total_dist/1000, avg_resp, critical_cov, comp_time[name]]

print("\n=== Métriques finales alignées ===")
print(metrics_df)


Nombre d'appels EMS utilisés : 10

=== Métriques finales alignées ===
     Total Distance (km)  Avg Response (min)  Critical Coverage (calls)  \
GA             67.383437           26.911770                        9.0   
TS             24.737280           12.606921                       10.0   
SA             63.158424           30.334532                        8.0   
ACO            51.489688           18.955174                       10.0   

     Computational Time (s)  
GA                 0.002371  
TS                 0.018567  
SA                 0.007145  
ACO                0.009259  
